In [15]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)
from rdflib import Graph, Namespace, URIRef
import import_ipynb
import FileWork

In [16]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [17]:
text_IT = open(r'C:\Users\Юлия\OneDrive\Документы\Универ\3 курс\8 триместр\Курсовая\Semantic annotation\files\text_IT.txt', encoding='UTF-8').read()
text_Friends = open(r'C:\Users\Юлия\OneDrive\Документы\Универ\3 курс\8 триместр\Курсовая\Semantic annotation\files\text_Friends.txt', encoding='UTF-8').read()

# print(f'Текст IT:\n{text_IT}\n\nТекст Fiends:\n{text_Friends}')

In [18]:
# db_path = r'C:\Users\Юлия\OneDrive\Документы\Универ\3 курс\8 триместр\Курсовая\Semantic annotation\files\ruwordnet-2021.db'
# FileWork.convert_db_to_owl(db_path)

In [19]:
ontology_graph = Graph()
ontology_graph.parse(r'C:\Users\Юлия\OneDrive\Документы\Универ\3 курс\8 триместр\Курсовая\Semantic annotation\files\my_ontologies.owl')
# ontology_graph.print()

<Graph identifier=N248d3c97c2974c77a629a4c4cb955394 (<class 'rdflib.graph.Graph'>)>

In [20]:
doc_IT = Doc(text_IT)
doc_Friends = Doc(text_Friends)

In [21]:
doc_IT.segment(segmenter)
doc_Friends.segment(segmenter)

In [22]:
doc_IT.tag_morph(morph_tagger)
doc_Friends.tag_morph(morph_tagger)

In [23]:
doc_IT.parse_syntax(syntax_parser)
doc_Friends.parse_syntax(syntax_parser)

In [24]:
for token in doc_IT.tokens:
    token.lemmatize(morph_vocab)

for token in doc_Friends.tokens:
    token.lemmatize(morph_vocab)

lemmas_doc_IT = []
lemmas_doc_Friends = []

for token in doc_IT.tokens:
    if token.pos not in {'PUNCT', 'PRED', 'CONJ', 'PART', 'CCONJ', 'SCONJ', 'ADP'}:
        lemmas_doc_IT.append(token.lemma.upper())

for token in doc_Friends.tokens:
    if token.pos not in {'PUNCT', 'PRED', 'CONJ', 'PART', 'CCONJ', 'SCONJ', 'ADP'}:
        lemmas_doc_Friends.append(token.lemma.upper())

print(lemmas_doc_IT)
print(lemmas_doc_Friends)

['КОМПАНИЯ', 'РЕШАТЬ', 'МНОЖЕСТВО', 'ПРОБЛЕМА', 'ИТ-СФЕРА', 'ОНА', 'ОБЪЕДИНЯТЬ', 'ЭКСПЕРТ', 'СОЗДАНИЕ', 'ИННОВАЦИОННЫЙ', 'РЕШЕНИЕ', 'ОНА', 'ВАЖНЫЙ', 'СИНХРОНИЗИРОВАТЬ', 'УСИЛИЕ', 'КОМАНДА', 'ДОСТИГНУТЬ', 'ПОСТАВИТЬ', 'ЦЕЛЬ', 'КАЖДЫЙ', 'СОТРУДНИК', 'КЛЮЧЕВОЙ', 'ЗВЕНО', 'ВНОСИТЬ', 'СВОЙ', 'ВКЛАД', 'УСПЕХ', 'РАБОТА', 'КОМПАНИЯ', 'ТРЕБОВАТЬ', 'ТЕХНИЧЕСКИЙ', 'НАВЫК', 'УМЕНИЕ', 'РАБОТАТЬ', 'КОЛЛЕКТИВ', 'ВМЕСТЕ', 'МЫ', 'СОЗДАВАТЬ', 'ПРОДУКТ', 'МЕНЯТЬ', 'МИР', 'КОМПАНИЯ', 'РАБОЧИЙ', 'МЕСТО', 'СООБЩЕСТВО', 'ЕДИНОМЫШЛЕННИК', 'СТРЕМИТЬСЯ', 'СОВЕРШЕНСТВО', 'ИТ']
['КОМПАНИЯ', 'ДРУГ', 'КАЖДЫЙ', 'ВСТРЕЧА', 'ПРАЗДНИК', 'МЫ', 'ДЕЛИТЬСЯ', 'РАДОСТЬ', 'СМЕХ', 'ИСТОРИЯ', 'ВМЕСТЕ', 'МЫ', 'ПРОВОДИТЬ', 'ВРЕМЯ', 'ПРИРОДА', 'ГОТОВИТЬ', 'ВКУСНЫЙ', 'БЛЮДО', 'УСТРАИВАТЬ', 'ПИКНИК', 'ПРОДУКТ', 'КОТОРЫЙ', 'МЫ', 'ВЫБИРАТЬ', 'ЛЮБОВЬ', 'СТАНОВИТЬСЯ', 'ЧАСТЬ', 'НАШ', 'ОБЩИЙ', 'СТОЛ', 'ФРУКТ', 'СЫР', 'КАЖДЫЙ', 'ВНОСИТЬ', 'СВОЙ', 'ВКЛАД', 'ТАКОЙ', 'АТМОСФЕРА', 'ДРУЖБА', 'КРЕПНУТЬ', 'ВОСПОМИНАНИЕ', 'СТАНОВИТЬСЯ', 'ЯРЧЕ', '

In [25]:
def get_ontologies(lemmas):
    ontologies_word = {}
    for word in lemmas:
        ontologies_word[word] = []
        synsets = ontology_graph.query(f"""
            SELECT ?synset_id
            WHERE {{
                ?sense rdf:type ns1:sense ;
                    ns1:name ?name ;
                    ns1:lemma ?lemma ;
                    ns1:synset_id ?synset_id .
                FILTER(CONTAINS(?lemma, "{word}"))
            }}
        """)
        for row in synsets:
            synset_id = row['synset_id']

            definitions = ontology_graph.query(f"""
                SELECT ?definition
                WHERE {{
                    ?synset rdf:type ns1:synset ;
                            ns1:id "{synset_id}" ;
                            ns1:definition ?definition .
                }}
            """)

            for d in definitions:
                ontologies_word[word].append(d['definition'])
    return ontologies_word

In [26]:
onologies_doc_IT = get_ontologies(lemmas_doc_IT)
onologies_doc_IT

{'КОМПАНИЯ': [rdflib.term.Literal('группа лиц, проводящих время совместно'),
  rdflib.term.Literal('компания, осуществляющая свою деятельность в области информационных технологий')],
 'РЕШАТЬ': [rdflib.term.Literal('обдумывать ситуацию, формулируя определённый вывод, делая заключение или вырабатывая план какого-то действия')],
 'МНОЖЕСТВО': [],
 'ПРОБЛЕМА': [rdflib.term.Literal('сложный теоретический или практический вопрос, требующий решения, исследования')],
 'ИТ-СФЕРА': [],
 'ОНА': [],
 'ОБЪЕДИНЯТЬ': [],
 'ЭКСПЕРТ': [rdflib.term.Literal('высококлассный специалист в какой-либо предметной области')],
 'СОЗДАНИЕ': [rdflib.term.Literal('создание, творение, плод творчества художника')],
 'ИННОВАЦИОННЫЙ': [],
 'РЕШЕНИЕ': [rdflib.term.Literal('обдумывать ситуацию, формулируя определённый вывод, делая заключение или вырабатывая план какого-то действия'),
  rdflib.term.Literal('обдумав вопрос или исследовав проблему, найти нужный ответ')],
 'ВАЖНЫЙ': [],
 'СИНХРОНИЗИРОВАТЬ': [rdflib.term.Lit

In [27]:
onologies_doc_Friend = get_ontologies(lemmas_doc_Friends)
onologies_doc_Friend

{'КОМПАНИЯ': [rdflib.term.Literal('группа лиц, проводящих время совместно'),
  rdflib.term.Literal('компания, осуществляющая свою деятельность в области информационных технологий')],
 'ДРУГ': [rdflib.term.Literal('тот, с кем поддерживают близкие, приятельские отношения; друг, приятель, товарищ')],
 'КАЖДЫЙ': [],
 'ВСТРЕЧА': [rdflib.term.Literal('собрание, устраиваемое с целью знакомства, обсуждения чего-либо и\xa0т.\xa0п')],
 'ПРАЗДНИК': [rdflib.term.Literal('официально отмечаемый торжественный день, связанный с годовщиной радостного события или церковной традицией')],
 'МЫ': [rdflib.term.Literal('человек, имеющий одинаковые с кем-либо мысли, взгляды, убеждения')],
 'ДЕЛИТЬСЯ': [rdflib.term.Literal('рассказывая, сообщая кому-либо о чём-либо, уделять кому-либо от своих знаний, сведений'),
  rdflib.term.Literal('доставить радость кому-либо, вызвать сильное положительное чувство в ком-либо')],
 'РАДОСТЬ': [rdflib.term.Literal('доставить радость кому-либо, вызвать сильное положительное чув

In [28]:
triples_doc_IT = FileWork.get_triples(text_IT)
triples_doc_Friends = FileWork.get_triples(text_Friends)

print(triples_doc_IT)

[['компания', 'решать', 'множество'], ['решать', 'множество', 'проблема'], ['множество', 'проблема', 'ит-сфера'], ['проблема', 'ит-сфера', 'объединять'], ['ит-сфера', 'объединять', 'эксперт'], ['объединять', 'эксперт', 'создание'], ['эксперт', 'создание', 'инновационный'], ['создание', 'инновационный', 'решение'], ['инновационный', 'решение', 'важно'], ['решение', 'важно', 'синхронизировать'], ['важно', 'синхронизировать', 'усилие'], ['синхронизировать', 'усилие', 'команда'], ['усилие', 'команда', 'достигнуть'], ['команда', 'достигнуть', 'поставить'], ['достигнуть', 'поставить', 'цель'], ['поставить', 'цель', 'каждый'], ['цель', 'каждый', 'сотрудник'], ['каждый', 'сотрудник', 'ключевой'], ['сотрудник', 'ключевой', 'звено'], ['ключевой', 'звено', 'вносить'], ['звено', 'вносить', 'свой'], ['вносить', 'свой', 'вклад'], ['свой', 'вклад', 'успех'], ['вклад', 'успех', 'работа'], ['успех', 'работа', 'компания'], ['работа', 'компания', 'требовать'], ['компания', 'требовать', 'технический'], ['

In [29]:
print(triples_doc_Friends)

[['компания', 'друг', 'каждый'], ['друг', 'каждый', 'встреча'], ['каждый', 'встреча', 'это'], ['встреча', 'это', 'праздник'], ['это', 'праздник', 'делиться'], ['праздник', 'делиться', 'радость'], ['делиться', 'радость', 'смех'], ['радость', 'смех', 'история'], ['смех', 'история', 'вместе'], ['история', 'вместе', 'проводить'], ['вместе', 'проводить', 'время'], ['проводить', 'время', 'природа'], ['время', 'природа', 'готовить'], ['природа', 'готовить', 'вкусный'], ['готовить', 'вкусный', 'блюдо'], ['вкусный', 'блюдо', 'устраивать'], ['блюдо', 'устраивать', 'пикник'], ['устраивать', 'пикник', 'продукт'], ['пикник', 'продукт', 'который'], ['продукт', 'который', 'выбирать'], ['который', 'выбирать', 'любовь'], ['выбирать', 'любовь', 'становиться'], ['любовь', 'становиться', 'часть'], ['становиться', 'часть', 'наш'], ['часть', 'наш', 'общий'], ['наш', 'общий', 'стол'], ['общий', 'стол', 'фрукт'], ['стол', 'фрукт', 'сыр'], ['фрукт', 'сыр', 'каждый'], ['сыр', 'каждый', 'вносить'], ['каждый', 'в